# NEM Open Data Extraction Tool

**The Objective**: to create a tool that can easily extract open data from nemweb.com.au and assemble it into an analysis-ready format.

* For the intitial testing phase, extracted data will be stored as CSV files
* Once the data extraction pipeline is set up, we can experiment with SQL databases

In [1]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

### Creating a custom JSON sitemap

We require a dynamic way of navigating the NEM website to locate and extract resources. A JSON file that tracks the data files in a given location on the NEM website could be helpful... 

* Define data directories i.e. a function to add new directories to JSON
  * Perhaps have a *comments* field that allows user to insert comments about the data resource
* Each data directory file has an additional flag whether the data has been extracted or not - maybe a CSV file might work better... (it could also convert the timestamps to datetime to allow datatime queries)